CSI6160 Homework 1 Decision Tree Implementation

John Olijnyk & John Machado

1/31/2022

We will implment the ID3 algorithm used for decision trees and demonstrate how Entropy and Information Gain are used to construct the decision tree.

References:

-  https://www.youtube.com/watch?v=coOTEc-0OGw

-  https://towardsdatascience.com/entropy-how-decision-trees-make-decisions-2946b9c18c8

-  https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f

In [6]:
import pandas as pd #for manipulating the csv data
import numpy as np #for mathematical calculation

In [7]:
# PlayTennis.csv is the data set from our lecture and also found on Kaggle used as an example for decision tree implementation
# We will open the data set CSV file and examine its elements
# We see we have 14 entries

train_data_set = pd.read_csv("TrainPlayTennis.csv")
train_data_set

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [8]:
#first we need to calculate the entropy 

def calc_total_entropy(df, label, class_list):

    """
        E(S) = Sum (-pi * lg(pi)) where i is each class

        Parameters:
            df (DataFrame): a pandas dataframe/dataset
            label (string): name of the label / ouput column of the dataframe to be used for supervised learning
            class_list (list): unique classes of the label / output column 

        Returns:
            (float): calculated entropy of the whole dataframe
    """

    total_row = df.shape[0] #the total size of the dataset
    total_entropy = 0 #initialize entropy to zero for summantion
    
    for c in class_list: #for each class in the label output
        total_class_count = df[df[label] == c].shape[0] #number of times we see this class in data set
        total_class_entropy =- (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entropy += total_class_entropy #adding the class entropy to the total entropy of the dataset
    
    return total_entropy

In [9]:
#now lets calculate the total entropy of the data set

label_string='Play Tennis' #name of pandas label i.e. output column for the supervised learning
label_class_list = ['Yes', 'No'] #set of possible values of the label as seen from the data set above

total_entropy = calc_total_entropy(train_data_set, label_string, label_class_list )

#we see this value matches the value that is calculated by hand

In [10]:
# next we need to calculate the entropy for each feature with its corresponding values
# therefore we will only examine each subset of values of a given feature as it relates to the output lablel
# need to calcuilate the total entropy for the feature i.e. E(S)
# need to calculate the entropy of each class of a feature.  ex:  E(Outlook=Sunny), E(Outlook=Rainy), E(Outlook=Outlook)
    
def calc_entropy(df, label, class_list):

    """
        calculates entropy for each feature in a dataset

        Parameters:
            df (DataFrame): a pandas dataframe/dataset, which contains rows that has a specific value of a feature (Ex. Rows with only Outlook = Sunny)
            label (string): name of the label of the dataframe
            class_list (list): unique classes of the label

        Returns:
            (float): calculated entropy of the feature value of the dataframe 
    """

    class_count = df.shape[0] #the number of classes in the df
    entropy = 0 #set entropy to zero for summation
    
    for c in class_list:  #for each class in the label output
        label_class_count = df[df[label] == c].shape[0] #row count of class c in data set
        entropy_class = 0  #set the entropy class count for summation
        if label_class_count != 0:  #handle the case of zero labels for that feature class to avoid divide by zero
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

In [11]:
# here we demonstrate the algorithm by selecting a the Outlook feature with class of Sunny.  
# We create a data frame with only those rows where Outlook is sunny

feature_values = train_data_set[train_data_set["Outlook"] == "Sunny"]
feature_values

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
10,Sunny,Mild,Normal,Strong,Yes


In [12]:
#now calculate the entropy of Outlook is Sunny

calc_entropy(feature_values, label_string, label_class_list)

#this matches the hand calculated value

0.9709505944546686

In [13]:
# here we demonstrate the algorithm by selecting a the Outlook feature with class of Rain.  
# We create a data frame with only those rows where Outlook is sunny

feature_values = train_data_set[train_data_set["Outlook"] == "Rain"]
feature_values

,Outlook,Temperature,Humidity,Wind,Play Tennis
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
9,Rain,Mild,Normal,Weak,Yes
13,Rain,Mild,High,Strong,No


In [14]:
#now calculate the entropy of Outlook is Sunny

calc_entropy(feature_values, label_string, label_class_list)

#this matches the hand calculated value

0.9709505944546686

In [15]:
# here we demonstrate the algorithm by selecting a the Outlook feature with class of Overcaset.  
# We create a data frame with only those rows where Outlook is sunny.

feature_values = train_data_set[train_data_set["Outlook"] == "Overcast"]
feature_values

,Outlook,Temperature,Humidity,Wind,Play Tennis
2,Overcast,Hot,High,Weak,Yes
6,Overcast,Cool,Normal,Strong,Yes
11,Overcast,Mild,High,Strong,Yes
12,Overcast,Hot,Normal,Weak,Yes


In [16]:
#now calculate the entropy of Outlook is Sunny

calc_entropy(feature_values, label_string, label_class_list)

#this matches the hand calculated value
#here we note that this has entropy of 0 because all of the lable output values are of one value.  Here 'Yes'

0.0

In [17]:
#next we need to calcualte the information gain that each feature provides

def calc_info_gain(feature_name, df, label, class_list):

    """
        IG(Y,X) = E(Y) - E(Y|X)

        Parameters:
            feature_name (string): the name of the feature column that we want to find it’s information gain
            df (DataFrame): a pandas dataframe/dataset
            label (string): name of the label of the dataframe (=Play Tennis)
            class_list (list): unique classes of the label (=[Yes, No]).
        
        Returns:
        (float): calculated information gain of the feature (Ex. for Outlook, returns 0.247)
    """

    feature_value_list = df[feature_name].unique() #unqiue values of the feature
    total_row = df.shape[0]  #determine number of rows
    feature_info = 0.0 #set information gain to zero to start sumamtion
    
    for feature_value in feature_value_list:  #iterate over each feature value
        feature_value_data = df[df[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    info_gain = calc_total_entropy(df, label, class_list) - feature_info #calculating information gain by subtracting from total
    return info_gain

In [18]:
#now calcluate the information gain for Outlook
calc_info_gain("Outlook", train_data_set, label_string, label_class_list)
#we see this agrees with the hand calculation

0.24674981977443933

In [19]:
#now calcluate the information gain for Humidity
calc_info_gain("Humidity", train_data_set, label_string, label_class_list)
#we see this agrees with the hand calculation

0.15183550136234159

In [20]:
#now calcluate the information gain for Temperature
calc_info_gain("Temperature", train_data_set, label_string, label_class_list)
#we see this agrees with the hand calculation

0.02922256565895487

In [21]:
#now calcluate the information gain for Wind
calc_info_gain("Wind", train_data_set, label_string, label_class_list)
#we see this agrees with the hand calculation

0.04812703040826949

In [22]:
#to buld the tre we want the value with the most information gain to be used first.  
#from above analysis we know that will be outlook

#we create a function to systematically find the feature in the data set with highest information gain


#df: a pandas dataframe/dataset
#label: string, name of the label of the dataframe (=Play Tennis)
#class_list: list, unique classes of the label (=[Yes, No]).
#returns: string, the feature name


def find_most_informative_feature(df, label, class_list):
    feature_list = df.columns.drop(label) #label is not a feature, so dropping it so we can iterate over all columns
    #finding the feature names in the dataset;
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, df, label, class_list)
        print('Feature:', feature, '  Info Gain:',  feature_info_gain) #print for demonstration each iteration on feature
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature, max_info_gain

In [23]:
#here we demonstrate the fuction working to find the most information gain
#we see each iteration produces that features info gain
#finally we get the max info gain and the reature
most_info_feature, most_info_gain = find_most_informative_feature(train_data_set, label_string, label_class_list)
print('Most Info Gain Result:  ', most_info_feature, ' - ', most_info_gain)

Feature: Outlook   Info Gain: 0.24674981977443933
Feature: Temperature   Info Gain: 0.02922256565895487
Feature: Humidity   Info Gain: 0.15183550136234159
Feature: Wind   Info Gain: 0.04812703040826949
Most Info Gain Result:   Outlook  -  0.24674981977443933


In [24]:
#now we will create the trees
#first we will create sub trees given the information gain

#feature_name: string, the name of the feature that we want to add to tree and shrink dataset (Ex. Outlook)
#df: a pandas dataframe/dataset
#label: string, name of the label of the dataframe (=Play Tennis)
#class_list: list, unique classes of the label (=[Yes, No]).
#returns: (dictionary, dataframe), the tree node with it’s branches and the updated dataset

def generate_sub_tree(feature_name, max_info_gain, df, label, class_list):
    feature_value_count_dict = df[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = df[df[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of feature_value = count of class (pure class)
                tree[feature_value] = c #adding node to the tree
                df = df[df[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #should extend the node, so the branch is marked with ?
            
    return tree, df

In [25]:
#now to demonstrate we will pass in from the previous step the most information gain feature.  This happend to be Outlook
root, train_data_sub = generate_sub_tree(most_info_feature, most_info_gain, train_data_set, label_string, label_class_list)

In [26]:
#as we examine the returned tree we see that it shows outlook.  T
#we know that Overcast appears to allways be a Yes
#for Sunny and Rain we need more features to better predict the label so those branches 
#have a ? for further building of the tree
root

{'Sunny': '?', 'Overcast': 'Yes', 'Rain': '?'}

In [27]:
#we also return a subset of the data without the Feature used in the tree.  In this case it is Outlook.
#We do this as we will need to iterate with the next feature that provide the most information gain.
train_data_sub

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes
10,Sunny,Mild,Normal,Strong,Yes
13,Rain,Mild,High,Strong,No


In [28]:
def make_tree(root, prev_feature_value, df, label, class_list):
    if df.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature, max_info_gain = find_most_informative_feature(df, label, class_list) #most informative feature
        print('Max Info Feature:',max_info_feature)
        print('Max Info Gain:',max_info_gain)
        tree, df = generate_sub_tree(max_info_feature, max_info_gain, df, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = df[df[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset

In [29]:
#now we create our tree.
#we see each iteration as it looks at the information gain of the remaining features 
#to select the next feature to use
root={}
make_tree(root,None,train_data_set, label_string, label_class_list)

Feature: Outlook   Info Gain: 0.24674981977443933
Feature: Temperature   Info Gain: 0.02922256565895487
Feature: Humidity   Info Gain: 0.15183550136234159
Feature: Wind   Info Gain: 0.04812703040826949
Max Info Feature: Outlook
Max Info Gain: 0.24674981977443933
Feature: Outlook   Info Gain: 0.0
Feature: Temperature   Info Gain: 0.5709505944546686
Feature: Humidity   Info Gain: 0.9709505944546686
Feature: Wind   Info Gain: 0.01997309402197489
Max Info Feature: Humidity
Max Info Gain: 0.9709505944546686
Feature: Outlook   Info Gain: 0.0
Feature: Temperature   Info Gain: 0.01997309402197489
Feature: Humidity   Info Gain: 0.01997309402197489
Feature: Wind   Info Gain: 0.9709505944546686
Max Info Feature: Wind
Max Info Gain: 0.9709505944546686


In [30]:
root

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

In [31]:
#now we can construct our id3 algorithm function

#df: a pandas dataframe/dataset
#label: string, name of the label of the dataframe (=Play Tennis)
#returns: (nested) dictionary, the decision tree
def id3(df, label):
    df_copy = df.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated is set empty to start
    class_list = df_copy[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, df, label, class_list) #start calling recursion
    return tree

In [32]:
tree = id3(train_data_set,"Play Tennis")

Feature: Outlook   Info Gain: 0.24674981977443933
Feature: Temperature   Info Gain: 0.02922256565895487
Feature: Humidity   Info Gain: 0.15183550136234159
Feature: Wind   Info Gain: 0.04812703040826949
Max Info Feature: Outlook
Max Info Gain: 0.24674981977443933
Feature: Outlook   Info Gain: 0.0
Feature: Temperature   Info Gain: 0.5709505944546686
Feature: Humidity   Info Gain: 0.9709505944546686
Feature: Wind   Info Gain: 0.01997309402197489
Max Info Feature: Humidity
Max Info Gain: 0.9709505944546686
Feature: Outlook   Info Gain: 0.0
Feature: Temperature   Info Gain: 0.01997309402197489
Feature: Humidity   Info Gain: 0.01997309402197489
Feature: Wind   Info Gain: 0.9709505944546686
Max Info Feature: Wind
Max Info Gain: 0.9709505944546686


In [33]:
# here is the construct full tree

In [34]:
#tree: (nested) dictionary, a decision tree
#instance: series/dataframe row, a row of dataset. The row may not contain label.
#returns: Any datatype (Int or Float or String etc.) depending on the datatype of the class, the predicted classs

def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

In [35]:
#To evaluate the model i.e. the tree we need a labeled test dataset. Then after predicting we can calculate the difference of actual and predicted value in percentage.
#tree: (nested) dictionary, a decision tree
#test_data_m: a pandas dataframe/test dataset
#returns: float, the accuray of the tree

def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows(): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[index]) #predict the row
        if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
            correct_preditct += 1 #increase correct count
        else:
            wrong_preditct += 1 #increase incorrect count
    accuracy = correct_preditct / (correct_preditct + wrong_preditct) #calculating accuracy
    return accuracy

In [36]:
test_data_m = pd.read_csv("TestPlayTennis.csv") #importing the dataset from the disk
test_data_m.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Overcast,Mild,Normal,Weak,Yes
1,Sunny,Mild,Normal,Weak,Yes
2,Rain,Mild,Normal,Weak,Yes
3,Rain,Cool,High,Strong,No


In [37]:
evaluate(tree, test_data_m, 'Play Tennis') #evaluating the test dataset

1.0

In [38]:
test_row = test_data_m.iloc[3]
test_row

Outlook          Rain
Temperature      Cool
Humidity         High
Wind           Strong
Play Tennis        No
Name: 3, dtype: object

In [39]:
predict(tree,test_row)

'No'